In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score, precision_recall_curve, auc
from scipy.stats import entropy
from sklearn.utils import shuffle


In [2]:
import sys
sys.path.append('/home/cmottez/CS231N_Lightweight_Bias_Mitigation_Chest_Xray')

from metrics import compute_metrics, compute_kl_divergence_sex, compute_kl_divergence_age, compute_kl_divergence_race, compute_metrics_subcath, bias_table, bias_table_auprc


### Read the CSV

In [3]:
train = pd.read_pickle('../data/embeddings_chexpert/CNNs/chexpert_on_chexpert_train_with_embeddings_extracted.pkl')
valid = pd.read_pickle('../data/embeddings_chexpert/CNNs/chexpert_on_chexpert_valid_with_embeddings_extracted.pkl')
test = pd.read_pickle('../data/embeddings_chexpert/CNNs/chexpert_on_chexpert_test_with_embeddings_extracted.pkl')

In [4]:
test.head()

,gender,race,age,insurance,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,embeddings
0,1,0,1,1,1,1,0,1,0,1,0,0,0,1,0,0,0,1,"[0.0029132624622434378, 0.1020001769065857, 0...."
1,0,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"[0.0014348188415169716, 0.0543656125664711, 0...."
2,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"[0.001982336398214102, 0.040021587163209915, 0..."
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"[0.001741771469824016, 0.0560498870909214, 0.1..."
4,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,"[9.678312198957428e-05, 0.12247737497091293, 0..."


In [5]:
col = ['gender', 'race', 'age', 'Atelectasis',
       'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
       'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
       'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
       'Support Devices', 'embeddings']
train = train[col]
valid = valid[col]
test = test[col]

In [6]:
initial_size = test.shape[0] 
test = test[test['embeddings'].apply(type) == list]
final_size = test.shape[0] 
print(f'Number of test removed rows = {initial_size - final_size}')

initial_size = train.shape[0] 
train = train[train['embeddings'].apply(type) == list]
final_size = train.shape[0] 
print(f'Number of train removed rows = {initial_size - final_size}')

Number of test removed rows = 51
Number of train removed rows = 67


### Train test

In [7]:
train_embeddings = pd.DataFrame(train['embeddings'].tolist())
test_embeddings = pd.DataFrame(test['embeddings'].tolist())

# Diseases to predict
diseases = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
            'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture']
# diseases = ['Cardiomegaly', 'Lung Opacity', 'Edema', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']
diseases = ['Cardiomegaly', 'Lung Opacity', 'Edema', 'Pleural Effusion']
diseases = ['Pleural Effusion']
# diseases = ['gender']

# Labels for train and test
y_train = train[diseases]
y_test = test[diseases]

X_train = train_embeddings
X_test = test_embeddings


In [8]:
y_sex = test['gender']
y_race = test['race']
y_age = test['age']


In [9]:
y_train.replace(-1, 0, inplace=True)
y_test.replace(-1, 0, inplace=True)

/tmp/ipykernel_3074475/3958011541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train.replace(-1, 0, inplace=True)
/tmp/ipykernel_3074475/3958011541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test.replace(-1, 0, inplace=True)


In [10]:
!pip install shap

In [11]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import shap
from scipy.stats import ttest_ind, f_oneway, spearmanr

# ——— 1. Train on full data ———
clf = LogisticRegression(
    max_iter=1000, 
    solver='liblinear', 
    random_state=0
)
clf.fit(X_train, y_train.values.ravel())

# ——— 2. Compute SHAP with the new API ———
explainer = shap.Explainer(clf, X_train)
shap_exp  = explainer(X_test)

# shap_exp.values is now (n_samples, n_features)
shap_vals = shap_exp.values
print("SHAP array shape:", shap_vals.shape, "vs X_test shape:", X_test.shape)

# ——— 3. Build DataFrame of SHAP + subgroup labels ———
assert shap_vals.shape[1] == X_test.shape[1], "Mismatch in #features!"
shap_df = pd.DataFrame(shap_vals, columns=X_test.columns)
shap_df['gender'] = y_sex.values
shap_df['race']   = y_race.values
shap_df['age']    = y_age.values

# ——— 4. Statistical comparisons ———

# 4a) Gender (binary t‐test)
g0, g1 = shap_df['gender'].unique()[:2]
p_gender = {
    feat: ttest_ind(
             shap_df.loc[shap_df['gender']==g0, feat],
             shap_df.loc[shap_df['gender']==g1, feat],
             equal_var=False
           ).pvalue
    for feat in X_test.columns
}
print("Top gender‐differentiated features:")
print(pd.Series(p_gender).nsmallest(10))

# 4b) Race (one‐way ANOVA)
races = shap_df['race'].unique()
p_race = {}
for feat in X_test.columns:
    groups = [shap_df.loc[shap_df['race']==r, feat] for r in races]
    p_race[feat] = f_oneway(*groups).pvalue
print("\nTop race‐differentiated features:")
print(pd.Series(p_race).nsmallest(10))

# 4a) Age (binary t‐test)
g0, g1 = shap_df['age'].unique()[:2]
p_gender = {
    feat: ttest_ind(
             shap_df.loc[shap_df['age']==g0, feat],
             shap_df.loc[shap_df['age']==g1, feat],
             equal_var=False
           ).pvalue
    for feat in X_test.columns
}
print("Top age‐differentiated features:")
print(pd.Series(p_gender).nsmallest(10))


SHAP array shape: (40307, 1024) vs X_test shape: (40307, 1024)
Top gender‐differentiated features:
1      0.0
13     0.0
23     0.0
43     0.0
56     0.0
60     0.0
78     0.0
90     0.0
99     0.0
105    0.0
dtype: float64


Each of the input arrays is constant;the F statistic is not defined or infinite



Top race‐differentiated features:
238     0.000000e+00
293     0.000000e+00
304     0.000000e+00
435     0.000000e+00
312    1.482197e-323
475    6.200524e-321
172    8.692097e-320
10     1.427864e-313
316    2.530774e-292
170    3.434928e-291
dtype: float64
Top age‐differentiated features:
63     0.0
66     0.0
83     0.0
98     0.0
117    0.0
120    0.0
160    0.0
165    0.0
182    0.0
202    0.0
dtype: float64


In [12]:
import numpy as np
import pandas as pd

def direction_by_group(shap_df, subgroup_col, features):
    """
    Returns for each feature whether its mean SHAP sign is the same
    across all levels of subgroup_col, or flips in at least one.
    
    shap_df: DataFrame, columns = features + subgroup_col
    subgroup_col: e.g. 'gender' or 'race'
    features: list of feature names to check
    """
    # 1) Compute mean SHAP (signed) per group
    mean_shap = shap_df.groupby(subgroup_col)[features].mean()
    # 2) Extract the sign of each mean: +1, 0, or -1
    sign_df = mean_shap.apply(np.sign)
    
    result = {}
    for feat in features:
        signs = sign_df[feat].unique()
        # remove any zeros (exactly zero is rare, but treat as neutral)
        signs = set(s for s in signs if s != 0)
        if len(signs) <= 1:
            result[feat] = 'same direction'
        else:
            result[feat] = 'opposite direction'
    return pd.Series(result)


In [13]:
import pandas as pd

# ——— assume you already have shap_df with feature columns + 'gender' ———
features = X_test.columns

# 1) Compute mean(|SHAP|) per feature per gender
mean_abs = (
    shap_df
    .groupby('gender')[features]
    .apply(lambda df: df.abs().mean())
)
# `mean_abs` is now a DataFrame indexed by gender, columns = features

# 2) Sort and grab top‑N for each gender
top_n = 5
g0, g1 = mean_abs.index[:2]   # e.g. 'M' and 'F'
male_ranked   = mean_abs.loc[g0].sort_values(ascending=False)
female_ranked = mean_abs.loc[g1].sort_values(ascending=False)

male_top    = male_ranked.iloc[:top_n].index.tolist()
female_top  = female_ranked.iloc[:top_n].index.tolist()

print(f"Top {top_n} for {g0}:", male_top)
print(f"Top {top_n} for {g1}:", female_top)

# 3) Compute overlap and differences
set_m = set(male_top)
set_f = set(female_top)

common      = set_m & set_f
male_only   = set_m - set_f
female_only = set_f - set_m

print(f"\nCommon features ({len(common)}):", sorted(common))
print(f"{g0}-only ({len(male_only)}):", sorted(male_only))
print(f"{g1}-only ({len(female_only)}):", sorted(female_only))


Top 5 for 0: [773, 950, 781, 696, 603]
Top 5 for 1: [950, 696, 603, 773, 781]

Common features (5): [603, 696, 773, 781, 950]
0-only (0): []
1-only (0): []


In [14]:
top_union_gender = list(set(male_top) | set(female_top))

direction_gender = direction_by_group(shap_df, 'gender', top_union_gender)
print("Gender direction comparison:")
print(direction_gender)


Gender direction comparison:
773        same direction
781    opposite direction
950    opposite direction
696        same direction
603        same direction
dtype: object


In [15]:
import pandas as pd

# ——— assume you already have shap_df with feature columns + 'gender' ———
features = X_test.columns

# 1) Compute mean(|SHAP|) per feature per gender
mean_abs = (
    shap_df
    .groupby('age')[features]
    .apply(lambda df: df.abs().mean())
)
# `mean_abs` is now a DataFrame indexed by gender, columns = features

# 2) Sort and grab top‑N for each gender
top_n = 5
g0, g1 = mean_abs.index[:2]   # e.g. 'M' and 'F'
young_ranked   = mean_abs.loc[g0].sort_values(ascending=False)
old_ranked = mean_abs.loc[g1].sort_values(ascending=False)

young_top    = young_ranked.iloc[:top_n].index.tolist()
old_top  = old_ranked.iloc[:top_n].index.tolist()

print(f"Top {top_n} for {g0}:", young_top)
print(f"Top {top_n} for {g1}:", old_top)

# 3) Compute overlap and differences
set_y = set(young_top)
set_o = set(old_top)

common      = set_y & set_o
young_only   = set_y - set_o
old_only = set_o - set_y

print(f"\nCommon features ({len(common)}):", sorted(common))
print(f"{g0}-only ({len(young_only)}):", sorted(young_only))
print(f"{g1}-only ({len(old_only)}):", sorted(old_only))


Top 5 for 0: [950, 696, 773, 603, 781]
Top 5 for 1: [773, 950, 603, 781, 696]

Common features (5): [603, 696, 773, 781, 950]
0-only (0): []
1-only (0): []


In [16]:
top_union_age = list(set(young_top) | set(old_top))

direction_age = direction_by_group(shap_df, 'age', top_union_age)
print("Age direction comparison:")
print(direction_age)


Age direction comparison:
773        same direction
781        same direction
950    opposite direction
696    opposite direction
603    opposite direction
dtype: object


In [17]:
import pandas as pd
import itertools

# ——— assume shap_df has your SHAP values + a 'race' column ———
features = X_test.columns
top_n   = 5

# 1) Mean absolute SHAP per feature per race
mean_abs_race = (
    shap_df
    .groupby('race')[features]
    .apply(lambda df: df.abs().mean())
)
# mean_abs_race: index=race values, columns=features

# 2) Get top-N for each race
race_top = {}
for race in mean_abs_race.index:
    ranked = mean_abs_race.loc[race].sort_values(ascending=False)
    race_top[race] = ranked.iloc[:top_n].index.tolist()

# 3) Compare across races
sets = {r: set(flist) for r, flist in race_top.items()}

# 3a) Features common to ALL three races
common_all = set.intersection(*sets.values())

# 3b) Features common to exactly two races
common_pairs = {}
for (r1, r2) in itertools.combinations(sets.keys(), 2):
    common_pairs[(r1, r2)] = sets[r1] & sets[r2] - common_all

# 3c) Features unique to each race
unique_per_race = {
    r: sets[r] - set.union(*(sets[r2] for r2 in sets if r2 != r))
    for r in sets
}

# 4) Print results
print(f"Top {top_n} per race:")
for r, flist in race_top.items():
    print(f" Race {r}: {flist}")

print("\nCommon to all three races:", sorted(common_all))
print("\nCommon to exactly two races:")
for pair, feats in common_pairs.items():
    print(f" Races {pair}: {sorted(feats)}")

print("\nUnique per race:")
for r, feats in unique_per_race.items():
    print(f" Race {r}-only: {sorted(feats)}")


Top 5 per race:
 Race 0: [773, 950, 696, 603, 781]
 Race 1: [950, 696, 603, 773, 781]
 Race 2: [950, 696, 773, 603, 781]

Common to all three races: [603, 696, 773, 781, 950]

Common to exactly two races:
 Races (0, 1): []
 Races (0, 2): []
 Races (1, 2): []

Unique per race:
 Race 0-only: []
 Race 1-only: []
 Race 2-only: []


In [18]:
top_union_race = list(set(race_top[0]) | set(race_top[1]) | set(race_top[2]))

direction_race = direction_by_group(shap_df, 'race', top_union_race)
print("\nRace direction comparison:")
print(direction_race)



Race direction comparison:
773        same direction
781        same direction
950    opposite direction
696    opposite direction
603        same direction
dtype: object
